1. Preprocessing and Tokenization
Before performing fuzzy matching, preprocess and tokenize the names to standardize the format. This can include:

Lowercasing all names.
Removing punctuation and special characters.
Splitting names into tokens.
2. Indexing for Efficient Lookups
Use an indexing method to narrow down the number of comparisons:

Trie (Prefix Tree): Efficiently store and search large sets of strings. This can quickly eliminate many non-matching names.
Inverted Index: Create an inverted index where each token points to the list of names containing that token. This can significantly reduce the search space.
3. Approximate Nearest Neighbors (ANN)
Use an ANN algorithm for efficient similarity search:

FAISS (Facebook AI Similarity Search): An efficient library for ANN search. It can handle millions of names and provide fast approximate nearest neighbor lookups.
Annoy (Approximate Nearest Neighbors Oh Yeah): Another efficient library for ANN search, suitable for large datasets.
4. Combining Fuzzy Matching Methods
Use a combination of fuzzy matching methods to compute similarity scores and identify the best matches.

Preprocess and tokenize names to standardize format.
Index the names using a trie or inverted index for efficient lookups.
Use ANN algorithms (FAISS or Annoy) to narrow down the search space.
Apply combined fuzzy matching methods to determine the best match from the reduced candidate list.

In [4]:
import re
import numpy as np
from rapidfuzz import fuzz
from annoy import AnnoyIndex

# Preprocessing function
def preprocess_name(name):
    name = name.lower()
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    tokens = name.split()
    tokens.sort()  # Sort tokens for token_sort_ratio
    return ' '.join(tokens)



In [5]:
# Preprocess the entire corpus
corpus_of_names = ["John Doe", "Jane Smith", "Alice Johnson"]  # Add all million names here
preprocessed_names = [preprocess_name(name) for name in corpus_of_names]

# Build Annoy index
embedding_dim = 128  # Dimensionality of the embeddings
annoy_index = AnnoyIndex(embedding_dim, 'angular')

# Convert names to vectors (simple bag-of-words vectorization)
def name_to_vector(name):
    vector = np.zeros(embedding_dim)
    tokens = name.split()
    for token in tokens:
        for char in token:
            vector[ord(char) % embedding_dim] += 1  # Simple hashing of characters
    return vector

name_vectors = [name_to_vector(name) for name in preprocessed_names]

for i, vector in enumerate(name_vectors):
    print(i, vector)
    annoy_index.add_item(i, vector)

# Build the index (number of trees can be adjusted for accuracy/speed trade-off)
annoy_index.build(10)  # 10 trees

0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 2. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
2 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

True

In [10]:
# Fuzzy matching score function
function_selection = {
    "ratio": 1,
    "partial_ratio": 0,
    "token_sort_ratio": 1,
    "token_set_ratio": 1,
    "partial_token_sort_ratio": 0,
    "partial_token_set_ratio": 0
}
def combined_fuzzy_score(name1, name2):
    ratio = fuzz.ratio(name1, name2) * function_selection["ratio"]
    partial_ratio = fuzz.partial_ratio(name1, name2) * function_selection["partial_ratio"]
    token_sort_ratio = fuzz.token_sort_ratio(name1, name2)  * function_selection["token_sort_ratio"]
    token_set_ratio = fuzz.token_set_ratio(name1, name2)    * function_selection["token_set_ratio"]
    partial_token_sort_ratio = fuzz.partial_token_sort_ratio(name1, name2) * function_selection["partial_token_sort_ratio"]
    partial_token_set_ratio = fuzz.partial_token_set_ratio(name1, name2) * function_selection["partial_token_set_ratio"]
    print(f"ratio: {ratio}, partial_ratio: {partial_ratio}, token_sort_ratio: {token_sort_ratio}, token_set_ratio: {token_set_ratio}, partial_token_sort_ratio: {partial_token_sort_ratio}, partial_token_set_ratio: {partial_token_set_ratio}")
    # print(f"Ratios: {ratio}, {partial_ratio}, {token_sort_ratio}, {token_set_ratio}, {partial_token_sort_ratio}, {partial_token_set_ratio}")
    return sum([ratio, partial_ratio, token_sort_ratio, token_set_ratio, partial_token_sort_ratio, partial_token_set_ratio])/sum(function_selection.values())
    return max(ratio, partial_ratio, token_sort_ratio, token_set_ratio, partial_token_sort_ratio, partial_token_set_ratio)

In [11]:
# Function to find best match
def find_best_match(input_name, num_candidates=10):
    preprocessed_input = preprocess_name(input_name)
    input_vector = name_to_vector(preprocessed_input)
    
    # Retrieve nearest neighbors using Annoy
    nearest_neighbors = annoy_index.get_nns_by_vector(input_vector, num_candidates, include_distances=False)
    print(nearest_neighbors)
    best_match_score = 0
    best_match_name = None
    
    for neighbor_idx in nearest_neighbors:
        candidate = preprocessed_names[neighbor_idx]
        score = combined_fuzzy_score(preprocessed_input, candidate)
        if score > best_match_score:
            best_match_score = score
            best_match_name = corpus_of_names[neighbor_idx]
    
    return best_match_name, best_match_score


In [12]:
# Example usage
input_name = "John A. Doe"
best_match, score = find_best_match(input_name)
print(f"Best match: {best_match} with score {score}")

[0, 2, 1]
ratio: 88.88888888888889, partial_ratio: 0.0, token_sort_ratio: 88.88888888888889, token_set_ratio: 100.0, partial_token_sort_ratio: 0.0, partial_token_set_ratio: 0.0
ratio: 60.86956521739131, partial_ratio: 0.0, token_sort_ratio: 60.86956521739131, token_set_ratio: 60.869565217391305, partial_token_sort_ratio: 0.0, partial_token_set_ratio: 0.0
ratio: 40.0, partial_ratio: 0.0, token_sort_ratio: 40.0, token_set_ratio: 40.0, partial_token_sort_ratio: 0.0, partial_token_set_ratio: 0.0
Best match: John Doe with score 92.5925925925926


Preprocessing:

The preprocess_name function normalizes the names by lowercasing and removing punctuation.
Names are then tokenized and sorted to handle different orderings.
Vectorization:

name_to_vector converts names into simple bag-of-words vectors based on character hashing.
Indexing with Annoy:

An Annoy index is built using the vectors of preprocessed names.
annoy_index.build builds the index with a specified number of trees. More trees increase accuracy at the cost of speed.
Fuzzy Matching:

combined_fuzzy_score combines several fuzzy matching methods from rapidfuzz to compute a comprehensive similarity score.
Finding the Best Match:

find_best_match preprocesses the input name, converts it to a vector, retrieves the nearest neighbors from the Annoy index, and then applies the fuzzy matching methods to find the best match.